# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load CSV to dataframe
city_data_path = "../Output/city_data.csv" 
city_data_df = pd.read_csv(city_data_path)
city_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saint George,90,US,1584157038,93,37.10,-113.58,48.99,3.36
1,Cape Town,75,ZA,1584156472,93,-33.93,18.42,60.01,8.05
2,Muros,33,ES,1584157163,93,42.78,-9.06,46.00,3.36
3,Albany,20,US,1584156902,52,42.60,-73.97,42.01,11.41
4,Jamestown,1,US,1584157168,59,42.10,-79.24,37.00,11.41
...,...,...,...,...,...,...,...,...,...
538,Codrington,80,AU,1584157804,67,-38.27,141.97,56.73,17.20
539,Solnechnyy,100,RU,1584157805,86,50.72,136.63,12.43,4.21
540,Oussouye,0,SN,1584157806,50,12.48,-16.55,78.80,6.93
541,Hukay,42,PH,1584157808,69,13.85,120.70,82.00,3.96


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#build map figure
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "TERRAIN")

In [4]:
#get lats and longs as lists
lat_list = city_data_df["Lat"].to_list()
long_list = city_data_df["Lng"].to_list()
#zip list into tuple pairs
coords_zip = zip(lat_list, long_list)
#turn zip object into normal list
coords_list = [coord for coord in coords_zip]

In [5]:
#build and add heatmap layer
humidity = city_data_df["Humidity"]
heat_layer = gmaps.heatmap_layer(coords_list, weights = humidity, max_intensity = max(humidity))
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#assign variables
temp = city_data_df["Max Temp"]
w_spd = city_data_df["Wind Speed"]
humdty = city_data_df["Humidity"]
clouds = city_data_df["Cloudiness"]

#grab values based on weather conditions   
vacay_df = city_data_df.loc[(clouds < 20) &
((temp <= 90) & (temp >= 70)) &
((humdty >= 35) & (humdty <= 60)) &
(w_spd <=15)]
vacay_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
86,Juan de Ayolas,3,PY,1584157263,47,-27.40,-56.90,83.05,3.87
119,Bubaque,0,GW,1584157303,60,11.28,-15.83,81.86,12.30
134,Gladstone,0,AU,1584157321,39,-23.85,151.25,84.20,11.41
224,Airai,12,TL,1584157431,45,-8.93,125.41,82.40,7.11
256,Ormara,0,PK,1584157467,58,25.21,64.64,71.22,2.91
384,Nicoya,3,CR,1584157473,58,10.15,-85.45,82.40,11.41
390,Moree,0,AU,1584157627,42,-29.47,149.85,82.40,8.05
393,Yeppoon,0,AU,1584157630,35,-23.13,150.73,86.00,13.87
426,Sittwe,0,MM,1584157669,54,20.15,92.90,81.45,3.87
540,Oussouye,0,SN,1584157806,50,12.48,-16.55,78.80,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#rename df and reset index
hotels_df = pd.DataFrame(vacay_df)
hotels_df = hotels_df.reset_index(drop=True)

In [15]:
hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Juan de Ayolas,3,PY,1584157263,47,-27.40,-56.90,83.05,3.87
1,Bubaque,0,GW,1584157303,60,11.28,-15.83,81.86,12.30
2,Gladstone,0,AU,1584157321,39,-23.85,151.25,84.20,11.41
3,Airai,12,TL,1584157431,45,-8.93,125.41,82.40,7.11
4,Ormara,0,PK,1584157467,58,25.21,64.64,71.22,2.91
5,Nicoya,3,CR,1584157473,58,10.15,-85.45,82.40,11.41
6,Moree,0,AU,1584157627,42,-29.47,149.85,82.40,8.05
7,Yeppoon,0,AU,1584157630,35,-23.13,150.73,86.00,13.87
8,Sittwe,0,MM,1584157669,54,20.15,92.90,81.45,3.87
9,Oussouye,0,SN,1584157806,50,12.48,-16.55,78.80,6.93


In [16]:
# define radius, base url, and list to keep results in
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
radius = 5000
hotel_name = []

#loop through df rows
for index in range(len(vacay_df)):
    #grab lat and longs
    lat = hotels_df.iloc[index, 5]
    long = hotels_df.iloc[index, 6]
    location = f"{lat}, {long}"
    #define query parameters
    params = {"query": "hotel",
              "location": location,
              "radius": radius,
              "key": g_key}
    #get response and append results to list
    response = requests.get(base_url, params=params).json()
    hotel_name.append(response["results"][1]["name"])

In [17]:
#add list to column in df
hotels_df["Hotel Name"] = hotel_name
hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Juan de Ayolas,3,PY,1584157263,47,-27.40,-56.90,83.05,3.87,Playa Municipal De Ayolas
1,Bubaque,0,GW,1584157303,60,11.28,-15.83,81.86,12.30,Bubaque Island Hotel
2,Gladstone,0,AU,1584157321,39,-23.85,151.25,84.20,11.41,The Club Hotel
3,Airai,12,TL,1584157431,45,-8.93,125.41,82.40,7.11,Pousada Alecrim Namrau
4,Ormara,0,PK,1584157467,58,25.21,64.64,71.22,2.91,Hafth Talar Beach Resort Ormara
5,Nicoya,3,CR,1584157473,58,10.15,-85.45,82.40,11.41,Rancho Humo Estancia
6,Moree,0,AU,1584157627,42,-29.47,149.85,82.40,8.05,Burke & Wills Motor Inn
7,Yeppoon,0,AU,1584157630,35,-23.13,150.73,86.00,13.87,Coral Inn Boutique Hotel
8,Sittwe,0,MM,1584157669,54,20.15,92.90,81.45,3.87,Hotel Memory
9,Oussouye,0,SN,1584157806,50,12.48,-16.55,78.80,6.93,Hôtel Katakalousse


In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…